AMPL book Chapter 1

Section 1.6 Adding Resource Constraint

In [35]:
from amplpy import AMPL, Environment
ampl = AMPL(Environment('D:\\amplide.mswin64\\amplide.mswin64'))

In [36]:
#ampl.option['solver'] = 'cplexamp'
#we can do this to set the solver we want.

In [37]:
ampl.read('./Model4.mod')

In [38]:
ampl.readData('./Model4.dat')

In [58]:
print(ampl.getObjective('Total_Profit').get())
print(ampl.getConstraint('Time').get('reheat'))
print(ampl.getConstraint('Time').get('roll'))

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];
subject to Time['reheat']:
	0.005*X['bands'] + 0.005*X['coils'] + 0.005*X['plate'] <= 35;
subject to Time['roll']:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;


In [40]:
ampl.eval('expand Total_Profit;')
ampl.eval('expand Time;')

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];

subject to Time['reheat']:
	0.005*X['bands'] + 0.005*X['coils'] + 0.005*X['plate'] <= 35;

subject to Time['roll']:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;



In [41]:
ampl.solve()

MINOS 5.51: optimal solution found.
4 iterations, objective 190071.4286


In [42]:
print(ampl.getObjective('Total_Profit').get().value())

190071.42857142858


##### Note that the profits have increased between TwoVariableLP-Model3.ipynb and ThreeVariableLP-Model3.ipynb by adding the new variable "plate".

In [43]:
ampl.eval('expand X["bands"];')
ampl.eval('expand X["coils"];')
ampl.eval('expand X["plate"];')

Coefficients of X['bands']:
	Time['reheat']   0.005
	Time['roll']     0.005
	Total_Profit    25

Coefficients of X['coils']:
	Time['reheat']   0.005
	Time['roll']     0.00714286
	Total_Profit    30

Coefficients of X['plate']:
	Time['reheat']   0.005
	Time['roll']     0.00625
	Total_Profit    29



In [44]:
ampl.eval('display _nvars, _ncons;')
#displays number of variables and number of constraints

_nvars = 3
_ncons = 2



#### If the system/problem number of constraints is much larger than number of variables; this will result in: "Problem has too few degrees of freedom".  There might be redundancy in these constraint. Use the command expand <Constraint_Name>; and check the constraints generated by AMPL. If "\_ncons" >> "\_nvars" then we may have misunderstood the requirements and unnecessarily defined constraints in the model.


In [45]:
print(ampl.getVariable('X').get('bands'))
print(ampl.getVariable('X').get('coils'))
print(ampl.getVariable('X').get('plate'))
print(ampl.getVariable('X').get('bands').value())
print(ampl.getVariable('X').get('coils').value())
print(ampl.getVariable('X').get('plate').value())

var X['bands'] >=1000, <=6000;
var X['coils'] >=500, <=4000;
var X['plate'] >=750, <=3500;
3357.142857142856
500.0
3142.857142857144


###### Notice how the min_limit > 0 and max_limit > 0. Therefore no production line can be halted. All products must be produced atleast upto the min_limit for each product. In the ThreeVariableLP-Model3b, the min_limit was 0 for all products. We can see how the objective cost function changes due to this and the values of the decision variables have shifted now. 

In [46]:
print(ampl.getData('X'))

   index0    |      X      
  'bands'    | 3357.142857142856
  'coils'    |     500     
  'plate'    | 3142.857142857144



In [47]:
values = ampl.getVariable('X').getValues()
print(values)

   index0    |    X.val    
  'bands'    | 3357.142857142856
  'coils'    |     500     
  'plate'    | 3142.857142857144



In [48]:
df = values.toPandas()
print(df)

             X.val
bands  3357.142857
coils   500.000000
plate  3142.857143


In [49]:
l = values.toList()
print(l)

[('bands', 3357.142857142856), ('coils', 500.0), ('plate', 3142.857142857144)]


In [50]:
d = values.toDict()
print(d)

{'bands': 3357.142857142856, 'coils': 500.0, 'plate': 3142.857142857144}


| TwoVariableLP                    | ThreeVariableLP           | ThreeVariableLP-Model3c           |
| :---                             |                           | ---:                         |
| Bands = 6000.0                   | Bands = 6000.0            | Bands = 6000.0            |
| Coils = 1399.9999999999993       | Coils =    0.0            | Coils =    500.0          |
| -                                | Plate = 1599.9999999999993| Plate = 1028.571429          |

In the ThreeVariableLP case, Coils went to 0.0 while Plate increased to 1599.99.
This makes sense because, profit per hour for :
*  bands = 200 tons/hour * 25 dollar/ton = 5000 dollar/hour
*  coils = 140 tons/hour * 30 dollar/ton = 4200 dollar/hour
*  plate = 160 tons/hour * 29 dollar/ton = 4640 dollar/hour

Based on the hourly profits, bands are the most profitable. So we make as much bands as possible but we are limited to make 6000 tons of bands. The next most profitable is plate, whatever remaining hours are available in the week, we make plate till we reach the maximum limit of 3500 tons of plate.

##### In the ThreeVariableLP-Model3c, Coils are produced at a minimum of 500, and Plate and production lowered to 1028.57.

ThreeVariableLP is these case were we consider only cost and try to attain the most profitable outcome. ThreeVariableLP-Model3c is the case where we need to maintain each product's production line because we may not be able to shut down production line for any of the products for a whole week.